In [1]:
import face_recognition
image = face_recognition.load_image_file("test.png")
face_locations = face_recognition.face_locations(image)

In [2]:
print(face_locations)

[(32, 487, 94, 425), (58, 121, 94, 85), (149, 494, 211, 432)]


In [9]:
import face_recognition
known_image = face_recognition.load_image_file("biden.png")
unknown_image = face_recognition.load_image_file("biden2.jpg")

biden_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces([biden_encoding], unknown_encoding)
face_landmarks_list = face_recognition.face_landmarks(image)

In [10]:
print(face_landmarks_list)

[{'chin': [(435, 48), (435, 55), (435, 63), (435, 70), (436, 77), (439, 85), (441, 92), (445, 99), (452, 102), (460, 101), (468, 97), (476, 91), (483, 85), (488, 77), (491, 69), (493, 60), (494, 50)], 'left_eyebrow': [(435, 45), (437, 42), (441, 42), (445, 43), (450, 45)], 'right_eyebrow': [(457, 46), (464, 45), (470, 46), (476, 48), (481, 52)], 'nose_bridge': [(453, 51), (451, 56), (450, 61), (449, 67)], 'nose_tip': [(445, 69), (447, 71), (450, 72), (454, 71), (457, 71)], 'left_eye': [(439, 49), (441, 47), (445, 48), (448, 51), (444, 51), (441, 50)], 'right_eye': [(463, 52), (466, 50), (470, 50), (473, 53), (470, 54), (466, 53)], 'top_lip': [(443, 80), (445, 79), (448, 78), (450, 79), (453, 79), (458, 80), (464, 82), (462, 82), (453, 81), (450, 81), (448, 81), (444, 80)], 'bottom_lip': [(464, 82), (458, 84), (453, 85), (450, 85), (447, 84), (445, 83), (443, 80), (444, 80), (448, 81), (450, 82), (453, 82), (462, 82)]}, {'chin': [(96, 69), (95, 73), (95, 77), (96, 80), (96, 84), (98, 88

In [14]:
import face_recognition
import cv2
import numpy as np

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(1)

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("majid.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("biden.png")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
]
known_face_names = [
    "Majid",
    "Joe Biden"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index] 
            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()